<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body>
    <div class="container">
        <h1>Basic Generative AI by Daniil Bokhan</h1>
        <h2>Based on stable-diffusion-xl-v1</h2>
        <p>Welcome to the Basic Generative AI project, created by Daniil Bokhan. This project utilizes the powerful stable-diffusion-xl-v1 model to generate high-quality images from textual descriptions.</p>
        <h3>About the Model</h3>
        <p>The stable-diffusion-xl-v1 model was released on June 3, 2023. It is designed to produce detailed and realistic images from text prompts, leveraging advanced diffusion techniques to enhance the quality and coherence of the generated images.</p>
        <p>This model represents a significant advancement in the field of generative AI, providing a versatile tool for artists, designers, and researchers to create stunning visuals with minimal input.</p>
        <h3>Key Features</h3>
        <ul>
            <li>High-quality image generation from text prompts</li>
            <li>Advanced noise reduction and refinement capabilities</li>
            <li>Supports a wide range of styles and subjects</li>
            <li>Efficient performance on modern hardware</li>
        </ul>
        <h3>How to Use</h3>
        <p>To use this application, simply enter a descriptive prompt in the input field and click the "Generate image" button. The AI model will process your request and produce an image based on your description.</p>
        <form method="POST" action="/">
            <input type="text" name="prompt" placeholder="Enter your request" required>
            <button type="submit">Generate image</button>
        </form>
        <h3>Project Overview</h3>
        <p>In this project, I will demonstrate how to create a simple website that processes text prompts and generates images using the stable-diffusion-xl-v1 model. This website will be built with minimal changes to the model and without requiring extensive code.</p>
        <p>The website will feature a straightforward user interface where users can input their prompts, submit them, and view the generated images. The backend will handle the processing of these prompts and the generation of images, showcasing the capabilities of the stable-diffusion-xl-v1 model in an accessible and user-friendly manner.</p>
    </div>
</body>
</html>


<!-- Image Divider -->
<p align="center">
    <img src="https://aplicaciones-ia.org/wp-content/uploads/2023/12/stability-logo.png"  width="100%">
</p>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body>
     <h1>script & model </h1>
</body>
</html>

In [ ]:
from flask import Flask, render_template, request, send_file, url_for
from diffusers import DiffusionPipeline
import torch
from io import BytesIO
import os

app = Flask(__name__)

def load_pipelines():
    base = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16"
    )
    refiner = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-refiner-1.0",
        text_encoder_2=base.text_encoder_2,
        vae=base.vae,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16"
    )
    return base.to(device), refiner.to(device)

device = "cuda" if torch.cuda.is_available() else "cpu"
base, refiner = load_pipelines()

def generate_image(prompt, base, refiner, n_steps=40, high_noise_frac=0.8):
    with torch.no_grad():
        image = base(prompt=prompt, num_inference_steps=n_steps, denoising_end=high_noise_frac, output_type="latent").images
        image = refiner(prompt=prompt, num_inference_steps=n_steps, denoising_start=high_noise_frac, image=image).images[0]
    return image

@app.route("/", methods=["GET", "POST"])
def index():
    image_url = None
    if request.method == "POST":
        prompt = request.form["prompt"]
        try:
            image = generate_image(prompt, base, refiner)
            img_io = BytesIO()
            image.save(img_io, 'PNG')
            img_io.seek(0)
            image_path = os.path.join('static', 'generated_image.png')
            with open(image_path, 'wb') as f:
                f.write(img_io.getbuffer())
            image_url = url_for('static', filename='generated_image.png')
        except Exception as e:
            return f"Error: {e}"
    return render_template("index.html", image_url=image_url)

if __name__ == "__main__":
    app.run(debug=True)

<a href="https://ibb.co/n0Z90yd"><img src="https://i.ibb.co/LPbmPyj/5.png" alt="5" border="0"></a>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body>
     <h1>html</h1>
</body>
</html>

In [ ]:
&lt;!DOCTYPE html&gt;
&lt;html lang=&quot;en&quot;&gt;
&lt;head&gt;
    &lt;meta charset=&quot;UTF-8&quot;&gt;
    &lt;meta name=&quot;viewport&quot; content=&quot;width=device-width, initial-scale=1.0&quot;&gt;
&lt;/head&gt;
&lt;body&gt;
    &lt;!-- Loader element --&gt;
    &lt;div class=&quot;loader&quot; id=&quot;global-loader&quot;&gt;&lt;/div&gt;
    &lt;div class=&quot;container&quot;&gt;
        &lt;!-- Title and description --&gt;
        &lt;h1&gt;Basic Generative AI by Daniil Bokhan&lt;/h1&gt;
        &lt;h2&gt;Based on stable-diffusion-xl&lt;/h2&gt;
        &lt;!-- Form to submit prompt --&gt;
        &lt;form method=&quot;POST&quot;&gt;
            &lt;input type=&quot;text&quot; name=&quot;prompt&quot; placeholder=&quot;Enter your request&quot; required&gt;
            &lt;button type=&quot;submit&quot; class=&quot;generate-btn&quot;&gt;Generate image&lt;/button&gt;
        &lt;/form&gt;
        &lt;!-- Image container --&gt;
        &lt;div class=&quot;image-container&quot;&gt;
            {% if image_url %}
                &lt;!-- Display generated image if available --&gt;
                &lt;h2&gt;Generated image:&lt;/h2&gt;
                &lt;div class=&quot;image-frame&quot;&gt;
                    &lt;img id=&quot;generated-image&quot; src=&quot;{{ image_url }}&quot; alt=&quot;generated image&quot; onload=&quot;hideLoader()&quot;&gt;
                &lt;/div&gt;
            {% endif %}
        &lt;/div&gt;
    &lt;/div&gt;

    &lt;!-- Theme switcher --&gt;
    &lt;div class=&quot;theme-switcher&quot;&gt;
        &lt;label for=&quot;theme-switch&quot;&gt;Dark Mode&lt;/label&gt;
        &lt;input type=&quot;checkbox&quot; id=&quot;theme-switch&quot;&gt;
    &lt;/div&gt;

    &lt;!-- Link to external JavaScript file --&gt;
    &lt;script src=&quot;{{ url_for('static', filename='script.js') }}&quot;&gt;&lt;/script&gt;
&lt;/body&gt;
&lt;/html&gt;


<a href="https://ibb.co/n0Z90yd"><img src="https://i.ibb.co/LPbmPyj/5.png" alt="5" border="0"></a>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body>
     <h1>css</h1>
</body>
</html>

In [ ]:
/* Base styles for the body */
body {
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
    background-color: #f4f4f4;
    color: #333;
    transition: background-color 0.3s, color 0.3s;
    position: relative;
}

/* Styles for the main content container */
.container {
    max-width: 600px;
    margin: 0 auto;
    padding: 20px;
    background-color: #fff;
    border-radius: 10px;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
    transition: background-color 0.3s;
    position: relative;
}

/* Heading styles */
h1, h2 {
    text-align: center;
    margin-bottom: 20px;
}

/* Form styles */
form {
    display: flex;
    flex-direction: column;
    align-items: center;
}

/* Input field styles */
input[type="text"] {
    width: 100%;
    padding: 10px;
    margin-bottom: 10px;
    border: 1px solid #ccc;
    border-radius: 5px;
}

/* Button styles */
button {
    padding: 10px 20px;
    background-color: #007bff;
    color: #fff;
    border: none;
    border-radius: 5px;
    cursor: pointer;
}

button:hover {
    background-color: #0056b3;
}

/* Styles for the image container */
.image-container {
    margin-top: 20px;
    text-align: center;
}

/* Styles for the image frame */
.image-frame {
    position: relative;
    margin-top: 20px;
    text-align: center;
    border: 2px solid #ccc;
    border-radius: 5px;
    overflow: hidden;
    display: inline-block;
}

/* Styles for the image */
.image-frame img {
    max-width: 100%;
    display: none;
}

/* Loader styles */
.loader {
    position: fixed;
    top: 50%;
    left: 50%;
    transform: translate(-50%, -50%);
    width: 50px;
    height: 50px;
    border: 5px solid #ccc;
    border-top: 5px solid #007bff;
    border-radius: 50%;
    animation: spin 1s linear infinite;
    z-index: 1000;
    display: none;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

/* Theme switcher styles */
.theme-switcher {
    position: fixed;
    bottom: 20px;
    right: 20px;
    z-index: 1000;
}

/* Dark mode styles */
body.dark-theme {
    background-color: #333;
    color: #fff;
}

body.dark-theme .container {
    background-color: #444;
    box-shadow: 0 0 10px rgba(255, 255, 255, 0.1);
}

body.dark-theme input[type="text"], body.dark-theme button {
    background-color: #555;
}

body.dark-theme .image-container {
    background-color: #666;
}

body.dark-theme .theme-switcher label {
    color: #fff;
}


<a href="https://ibb.co/n0Z90yd"><img src="https://i.ibb.co/LPbmPyj/5.png" alt="5" border="0"></a>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body>
     <h1>java script</h1>
</body>
</html>

In [ ]:
// Theme switcher functionality
const themeSwitch = document.getElementById('theme-switch');
themeSwitch.addEventListener('change', () => {
    // Toggle dark theme class on body
    document.body.classList.toggle('dark-theme');
    // Save theme preference to localStorage
    localStorage.setItem('darkThemeEnabled', themeSwitch.checked);
});

// Event listener for generate button
document.querySelector('.generate-btn').addEventListener('click', function() {
    // Display loader when generating image
    const loader = document.getElementById('global-loader');
    loader.style.display = 'block';
    // Hide previously generated image if any
    const image = document.getElementById('generated-image');
    if (image) {
        image.style.display = 'none';
    }
});

// Event listener for page load
window.addEventListener('load', () => {
    // Check for saved dark theme preference in localStorage
    const darkThemeEnabled = localStorage.getItem('darkThemeEnabled');
    // Apply dark theme if preference is true
    if (darkThemeEnabled && darkThemeEnabled === 'true') {
        document.body.classList.add('dark-theme');
        themeSwitch.checked = true;
    }
});

// Function to hide loader and display generated image
function hideLoader() {
    const loader = document.getElementById('global-loader');
    loader.style.display = 'none';
    const image = document.getElementById('generated-image');
    if (image) {
        image.style.display = 'block';
    }
}


<a href="https://ibb.co/n0Z90yd"><img src="https://i.ibb.co/LPbmPyj/5.png" alt="5" border="0"></a>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body>
     <h1>run the code and this is what the start screen looks like</h1>
</body>
</html>

<a href="https://ibb.co/wcq9gsk"><img src="https://i.ibb.co/QDV2Qdg/first.png" alt="first" border="0"></a>

<a href="https://ibb.co/n0Z90yd"><img src="https://i.ibb.co/LPbmPyj/5.png" alt="5" border="0"></a>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body>
     <h1>after writing the prompt</h1>
</body>
</html>

<a href="https://ibb.co/Ht8WDPG"><img src="https://i.ibb.co/tqr7M2Y/second.png" alt="second" border="0"></a>

<a href="https://ibb.co/n0Z90yd"><img src="https://i.ibb.co/LPbmPyj/5.png" alt="5" border="0"></a>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body>
     <h1>And...</h1>
</body>
</html>

<a href="https://ibb.co/RYypDy3"><img src="https://i.ibb.co/q75Mr5N/third.png" alt="third" border="0"></a>

<a href="https://ibb.co/n0Z90yd"><img src="https://i.ibb.co/LPbmPyj/5.png" alt="5" border="0"></a>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <h1>Conclusion</h1>
</head>
<body>
    <div class="container">
        <p>Approximate loading time on a 4060ti was about 7 minutes for 40 steps. The result is not perfect, there are remarks both on the picture and on the time, but still, I remind you that the model was not customized and it was released a year ago (at the time of writing). In my opinion, it's worth an hour of time, and you literally have a free image generator at your fingertips where you can make your funniest beaver or a cool boring car :) </p>
    </div>
</body>
</html>


<a href="https://ibb.co/n0Z90yd"><img src="https://i.ibb.co/LPbmPyj/5.png" alt="5" border="0"></a>

<h3>Additional</h3>
<h3>Technologies Used</h3>
<p><strong>Python:</strong></p>
<ul>
  <li>Flask: Used for creating the web application.</li>
  <li>Diffusers: Used for importing the DiffusionPipeline class for image processing.</li>
  <li>Torch: Used for working with neural networks and image processing.</li>
  <li>BytesIO: Used for working with bytes in memory.</li>
  <li>OS: Used for file and directory operations.</li>
</ul>

<p><strong>HTML:</strong></p>
<ul>
  <li>Used for webpage markup.</li>
</ul>

<p><strong>CSS:</strong></p>
<ul>
  <li>Used for styling the webpage and improving its appearance.</li>
</ul>

<p><strong>JavaScript:</strong></p>
<ul>
  <li>Used for interactivity and dynamic behavior of the webpage.</li>
</ul>

<p><strong>Jupyter Notebook:</strong></p>
<ul>
  <li>Used for code development, testing, and project documentation.</li>
</ul>

<h3>Thank You for Your Attention!</h3>
<p>If you find any errors in the code, please feel free to contact me at daniilbokhan.q@gmail.com. I'm eager to improve!</p>
